In [1]:
# Use this to complete all of my scraping/analysis tasks. 

from bs4 import BeautifulSoup as bs
import os
import requests
import pymongo
from splinter import Browser
import selenium # this does what? Selenium 3.14 is more recent than 3.5. 
import pandas as pd

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [10]:
# Need to scrape: https://mars.nasa.gov/news/   
# and collect the latest News Title and Paragraph Text. Assign the text to
# variables that I can reference later.

# e.g.: news_title = "NASA's Next Mars Mission to Investigate Interior"
    #   news_p = "Preparation of NASA's next spacecraft to Mars,
    #   InSight, has ramped up this summer, on course for launch next May 
    #   from Vandenberg Air Force Base in central California -- the first 
    #   interplanetary launch in history from America's West Coast."
    
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

html = browser.html
soup = bs(html, "html.parser")


news_title = soup.find_all("div", class_ = 'content_title')[1].text
print(news_title)

news_paragraph = soup.find_all("div", class_='article_teaser_body')[0].text
print(news_paragraph)

NASA Engineers Checking InSight's Weather Sensors
An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.


In [11]:
# Need to scrape: https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars

# Use Splinter to navigate the site and find the image url for the current 
# Featured Mars Image and assign the url string to a variable called 
# 'featured_image_url'. 
# e.g.: featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'

url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)

# Make sure to find the image url to the full (large) size '.jpg' image.
browser.click_link_by_partial_text('FULL IMAGE')
browser.click_link_by_partial_text('more info')

html = browser.html
soup = bs(html, "html.parser")

# Make sure to save a complete url string for this image. 

soup_image = soup.find("img", class_ ='main_image')

featured_image_url = f"https://www.jpl.nasa.gov/{soup_image.get('src')}"

print(featured_image_url)

/Users/haydenrubin/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/splinter/driver/webdriver/__init__.py:493: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA20057_hires.jpg


In [26]:
# Need to scrape the latest Mars weather tweet from: 
# https://twitter.com/marswxreport?lang=en . Save the tweet text for the
# weather report as a variable called 'mars_weather',
# e.g.: mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F,
# low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'

url3 = 'https://twitter.com/marswxreport?lang=en'

browser.visit(url3)

html = browser.html
soup = bs(html, "html.parser")

individ_img_url = soup.find("section", attrs={"aria-labelledby": "accessible-list-0"})

individ_img_url

#mars_weather = tweet.text

#print(mars_weather)

# print(tweet)

In [17]:
# Use Pandas to scrape the table containing facts about the planet
# including Diameter, Mass, etc. from https://space-facts.com/mars/

url4 = 'https://space-facts.com/mars/'

tables = pd.read_html(url4)
tables

# type(tables) = list. 

df = tables[0]
df.columns = ['Column 1', 'Column 2']

df.head()

# Use Pandas to convert the data to a HTML table string. 

html_table = df.to_html()
html_table.replace('\n', '')
df.to_html('Mars Facts Table.html')

In [4]:
url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url5)


In [22]:
# Need to scrape high resolutions images for each of Mar's hemispheres
# from https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars  .

html = browser.html
soup = bs(html, "html.parser")

# Need to click each of links to the hemispheres in order to find the
# image url to the full resolution image.

div_items = soup.find_all("div", class_ = 'item')
div_items

hemisphere_img_urls =  []

for item in div_items:
    title = item.find('h3').text # find, not find_all because it is a for loop.
    browser.click_link_by_partial_text(title)
    sample = browser.find_link_by_partial_text("Sample")
    html = browser.html
    soup = bs(html, "html.parser")

    hemi_url = sample[0]["href"]

    browser.back()
    
    valles_dict = {"title": title, "img_url": hemi_url}
    
    hemisphere_img_urls.append(valles_dict)

# Save both the image url string for the full resolution hemisphere
# image, and the Hemisphere title containing the hemisphere name. Use a 
# Python dictionary to store the data using the keys 'img_url' and 'title'.

# Append the dictionary with the image url string and the hemisphere
# title to a list. This list will contain one dictionary for each hemisphere

# e.g.:
# hemisphere_image_urls = [ {"title": "Valles Marineris Hemi",
# "img_url": "..."}, {"title": "Cerberus Hemisphere", "img_url": "..."},
# {"title": "Schiaparelli Hemisphere", "img_url": "..."}, {"title":
# "Syrtis Major Hemisphere", "img_url": "..."}]
hemisphere_img_urls


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]